In [7]:
from MIBooster import *
from load_spatialdata import *
from utils import *
import numpy as np

In [8]:
## Load the temperature data
data_train, data_val, data_test = get_air_temp_data()

print(data_train.head())

            y         x     meanT     meanP
1501  43.6064  -93.3019  0.492299  0.141646
1643  40.6742 -100.4936  0.550322  0.093253
1880  34.9894  -99.0525  0.683560  0.106676
3003  40.7300   31.6000  0.579513  0.100495
758   10.4800   99.1800  0.922815  0.344931


In [9]:
target = "meanT"
feature_cols = [col for col in data_train.columns if col != target]

X_train = data_train[feature_cols].values
y_train = data_train[target].values
locs_train = np.deg2rad(data_train[['y','x']].values)
X_val = data_val[feature_cols].values
y_val = data_val[target].values
locs_val = np.deg2rad(data_val[['y','x']].values)
X_test = data_test[feature_cols].values
y_test = data_test[target].values
locs_test = np.deg2rad(data_test[['y','x']].values)

In [10]:
mi_booster = MIXGBooster(k=35, verbosity=0)
mi_booster.fit(X_train, y_train, locs_train)
y_pred = mi_booster.predict(X_val)
print("RMSE: ", np.sqrt(np.mean((y_val-y_pred)**2)))

RMSE:  0.04062470652849581


In [11]:
mi_booster.w

<2153x2153 sparse matrix of type '<class 'numpy.float64'>'
	with 75355 stored elements in Compressed Sparse Row format>

In [12]:
from xgboost import XGBRegressor
xgb_baseline = XGBRegressor(verbosity=0)
xgb_baseline.fit(X_train, y_train)
y_pred = xgb_baseline.predict(X_val)
print("RMSE: ", np.sqrt(np.mean((y_val-y_pred)**2)))

RMSE:  0.041853329806057445
